In [2]:
import os
# 【硬性要求】加载环境变量
from dotenv import load_dotenv
load_dotenv(override=True)

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_classic.agents import create_tool_calling_agent, AgentExecutor
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# ==========================================
# 1. 初始化工具与大模型 (Init Tools & LLM)
# ==========================================
# 实例化 Tavily 搜索引擎
search_tool = TavilySearchResults()
tools = [search_tool]

# 初始化 LLM (作为 Agent 的大脑)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# ==========================================
# 2. 构建 Agent 核心提示词 (Build Agent Prompt)
# ==========================================
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Think step by step before responding."),
    # 历史记录插槽 (维持对话连贯性)
    MessagesPlaceholder(variable_name="chat_history"), 
    ("human", "{input}"),
    # 【核心】：智能体草稿本插槽 (维系推理-行动循环)
    MessagesPlaceholder(variable_name="agent_scratchpad"), 
])

# ==========================================
# 3. 组装 Agent 与执行器 (Assemble Agent & Executor)
# ==========================================
# 绑定大脑、工具和提示词
agent = create_tool_calling_agent(llm, tools, prompt)

# 启动引擎！verbose=True 强烈建议在科研调试时开启，你能清楚看到后台的思考过程
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# ==========================================
# 4. 注入记忆管理 (Inject Memory Management)
# ==========================================
# 【科研级保姆注释 - 架构升级】
# 旧做法：直接 lambda session_id: message_history。这会导致多用户数据串智。
# 现代安全做法：使用一个全局字典 store 来维护不同 session_id 的独立记录。
store = {}
def get_session_history(session_id: str) -> ChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 包装 Executor，赋予其管理上下文的能力
agent_with_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# ==========================================
# 5. 端到端测试 (Execution)
# ==========================================
if __name__ == "__main__":
    config = {"configurable": {"session_id": "lab_session_1"}}
    
    print("=== 测试 1: 日常问候 (测试记忆) ===")
    res1 = agent_with_history.invoke({"input": "hi, I'm a researcher!"}, config=config)
    print(f"🤖: {res1['output']}\n")
    
    print("=== 测试 2: 触发实时搜索工具 ===")
    # 问题：2024年 T20 世界杯什么时间举行？
    res2 = agent_with_history.invoke({"input": "When is the ICC Men's T20 2024 World Cup scheduled?"}, config=config)
    print(f"\n🤖: {res2['output']}\n")
    
    print("=== 测试 3: 揭示 LLM 的数学局限性 (The Cliffhanger) ===")
    # 老师讲稿中提到的局限性：大模型算不对距离开赛还有多少天
    res3 = agent_with_history.invoke({"input": "What's today's date? Based on today, exactly how many days are left before the first match starts?"}, config=config)
    print(f"\n🤖: {res3['output']}\n")

C:\Users\LIHAN\AppData\Local\Temp\ipykernel_7384\1557160633.py:17: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults()


=== 测试 1: 日常问候 (测试记忆) ===


> Entering new AgentExecutor chain...
Hello! It's great to have a researcher here. How can I assist you today?

> Finished chain.
🤖: Hello! It's great to have a researcher here. How can I assist you today?

=== 测试 2: 触发实时搜索工具 ===


> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': "ICC Men's T20 2024 World Cup schedule"}`


[{'title': "ICC Men's T20 Cricket World Cup 2024 full schedule", 'url': 'https://www.olympics.com/en/news/t20-cricket-world-cup-2024-schedule-results-scores-standings-full-list', 'content': "The 2024 ICC Men's T20 World Cup was co-hosted by the United States and West Indies in June.\n\nThe final – at the Kensington Oval in Barbados on 29 June – came to a nail-biting conclusion with India beating South Africa by seven runs to win their first World Cup title since the inaugural tournament in 2007.\n\nThe cricket tournament featured 20 teams for the first time, having expanded from 16, and were spl